In [56]:
import pandas as pd
import numpy as np

In [16]:
file_path_orders_payment = r'C:\Users\mathe\Projects\clients_seg_database\olist_order_payments_dataset.csv'
file_path_orders = r'C:\Users\mathe\Projects\clients_seg_database\olist_orders_dataset.csv'
file_path_customers = r'C:\Users\mathe\Projects\clients_seg_database\olist_customers_dataset.csv'

In [18]:
df_orders_payment = pd.read_csv(file_path_orders_payment)
df_orders = pd.read_csv(file_path_orders)
df_customers = pd.read_csv(file_path_customers)

In [20]:
df_orders_payment_selected = df_orders_payment[['order_id', 'payment_type', 'payment_installments', 'payment_value']]
df_orders_selected = df_orders[['order_id', 'customer_id', 'order_approved_at']]
df_customers_selected = df_customers[['customer_id', 'customer_unique_id']]

In [27]:
df_merge = pd.merge(
    df_orders_payment_selected,
    df_orders_selected,
    left_on='order_id',
    right_on='order_id',
    how='inner'
)

In [52]:
df_merge_2 = pd.merge(
    df_merge,
    df_customers_selected,
    left_on='customer_id',
    right_on='customer_id',
    how='inner'
)

In [53]:
df_merge_2['order_approved_at'] = pd.to_datetime(df_merge_2['order_approved_at'])
df_merge_2 = df_merge_2[df_merge_2['payment_type'] == 'credit_card']
df_merge_2 = df_merge_2[df_merge_2['order_approved_at'].notna()]

In [54]:
df_recencia_index = df_merge_2.groupby('customer_unique_id', as_index=False)['order_approved_at'].max()
df_recencia_index = df_recencia_index.sort_values('order_approved_at', ascending=False)
df_recencia_index['index'] = range(1, len(df_recencia_index) + 1)

In [57]:
condicoes = [
    (df_recencia_index['index'] <= (len(df_recencia_index) * 0.2)),
    (df_recencia_index['index'] > (len(df_recencia_index) * 0.2)) & (df_recencia_index['index'] <= (len(df_recencia_index) * 0.4)),
    (df_recencia_index['index'] > (len(df_recencia_index) * 0.4)) & (df_recencia_index['index'] <= (len(df_recencia_index) * 0.6)),
    (df_recencia_index['index'] > (len(df_recencia_index) * 0.6)) & (df_recencia_index['index'] <= (len(df_recencia_index) * 0.8)),
    (df_recencia_index['index'] > (len(df_recencia_index) * 0.8))
]

valores = [
    '5',  
    '4',  
    '3',
    '2',
    '1'    
]

df_recencia_index['recencia_score'] = np.select(condicoes, valores, default='Outros')

In [58]:
print(df_recencia_index)

                     customer_unique_id   order_approved_at  index  \
10638  24ac2b4327e25baf39f2119e4228976a 2018-08-29 15:10:26      1   
52921  b701bebbdf478f5500348f03aff62121 2018-08-29 15:05:22      2   
26702  5c58de6fb80e93396e2f35642666b693 2018-08-29 14:31:07      3   
35250  7a22d14aa3c3599238509ddca4b93b01 2018-08-29 12:35:17      4   
1181   0421e7a23f21e5d54efed456aedbc513 2018-08-29 11:24:02      5   
...                                 ...                 ...    ...   
37663  823c47d4abda1f8ce7568145f76c2b85 2016-10-04 10:26:40  74051   
39467  88bc832f104b6dd684a17dbb1b76e778 2016-10-04 10:25:46  74052   
39107  87776adb449c551e74c13fc34f036105 2016-10-04 10:19:23  74053   
53418  b8b8726af116a5cfb35b0315ecef9172 2016-10-04 10:18:57  74054   
5920   14359ea0c7a105749c0a56478825b015 2016-10-04 09:43:32  74055   

      recencia_score  
10638              5  
52921              5  
26702              5  
35250              5  
1181               5  
...              ... 